In [1]:
import os
import json
import openai

import data_utils

In [41]:
dataset = "cifar100"
prompt_type = "important"

In [2]:
openai.api_key = open(os.path.join("./", ".openai_api_key"), "r").read()[:-1]

In [4]:
prompts = {
    "important" : "List the most important features for recognizing something as a \"goldfish\":\n\n-bright orange color\n-a small, round body\n-a long, flowing tail\n-a small mouth\n-orange fins\n\nList the most important features for recognizing something as a \"beerglass\":\n\n-a tall, cylindrical shape\n-clear or translucent color\n-opening at the top\n-a sturdy base\n-a handle\n\nList the most important features for recognizing something as a \"{}\":",
    "superclass" : "Give superclasses for the word \"tench\":\n\n-fish\n-vertebrate\n-animal\n\nGive superclasses for the word \"beer glass\":\n\n-glass\n-container\n-object\n\nGive superclasses for the word \"{}\":",
    "around" : "List the things most commonly seen around a \"tench\":\n\n- a pond\n-fish\n-a net\n-a rod\n-a reel\n-a hook\n-bait\n\nList the things most commonly seen around a \"beer glass\":\n\n- beer\n-a bar\n-a coaster\n-a napkin\n-a straw\n-a lime\n-a person\n\nList the things most commonly seen around a \"{}\":"
}

#base_prompt = prompts[prompt_type]

In [5]:
print(data_utils.LABEL_FILES)

{'places365': 'data/categories_places365_clean.txt', 'imagenet': 'data/imagenet_classes.txt', 'cifar10': 'data/cifar10_classes.txt', 'cifar100': 'data/cifar100_classes.txt', 'cifar100_task0_3456': 'data/cifar100_task0_3456_classes.txt', 'cifar100_task1_3456': 'data/cifar100_task1_3456_classes.txt', 'cifar100_task2_3456': 'data/cifar100_task2_3456_classes.txt', 'cifar100_task3_3456': 'data/cifar100_task3_3456_classes.txt', 'cifar100_task4_3456': 'data/cifar100_task4_3456_classes.txt', 'cub': 'data/cub_classes.txt'}


In [42]:
cls_file = data_utils.LABEL_FILES[dataset]
with open(cls_file, "r") as f:
    classes = f.read().split("\n")

In [6]:
def get_feature(classes,base_prompt):
    feature_dict = {}

    for i, label in enumerate(classes):
        feature_dict[label] = set()
        print("\n", i, label)
        for _ in range(2):
            response = openai.Completion.create(
                  model="text-davinci-002",
                  prompt=base_prompt.format(label),
                  temperature=0.7,
                  max_tokens=256,
                  top_p=1,
                  frequency_penalty=0,
                  presence_penalty=0
                )
            #clean up responses
            features = response["choices"][0]["text"]
            features = features.split("\n-")
            features = [feat.replace("\n", "") for feat in features]
            features = [feat.strip() for feat in features]
            features = [feat for feat in features if len(feat)>0]
            features = set(features)
            feature_dict[label].update(features)
        feature_dict[label] = sorted(list(feature_dict[label]))
    return feature_dict

In [7]:
prompt_list=["important","superclass","around"]

In [8]:
for i in range(5):
    dataset='cifar100_task%d_3456'%(i)
    cls_file = data_utils.LABEL_FILES[dataset]
    with open(cls_file, "r") as f:
        classes = f.read().split("\n")
    for prompt_type in prompt_list:
        feature_dict=get_feature(classes,prompt_type)
        json_object = json.dumps(feature_dict, indent=4)
        with open("data/concept_sets/gpt3_init/gpt3_{}_{}.json".format(dataset, prompt_type), "w") as outfile:
            outfile.write(json_object)


 0 bear

 1 bee

 2 butterfly

 3 camel

 4 caterpillar

 5 chair

 6 elephant

 7 forest

 8 hamster

 9 lion

 10 motorcycle

 11 otter

 12 plates

 13 sea

 14 shark

 15 shrew

 16 spider

 17 tank

 18 train

 19 willow

 0 bear

 1 bee

 2 butterfly

 3 camel

 4 caterpillar

 5 chair

 6 elephant

 7 forest

 8 hamster

 9 lion

 10 motorcycle

 11 otter

 12 plates

 13 sea

 14 shark

 15 shrew

 16 spider

 17 tank

 18 train

 19 willow

 0 bear

 1 bee

 2 butterfly

 3 camel

 4 caterpillar

 5 chair

 6 elephant

 7 forest

 8 hamster

 9 lion

 10 motorcycle

 11 otter

 12 plates

 13 sea

 14 shark

 15 shrew

 16 spider

 17 tank

 18 train

 19 willow

 0 beaver

 1 bowls

 2 boy

 3 bridge

 4 castle

 5 cockroach

 6 couch

 7 dinosaur

 8 house

 9 keyboard

 10 lawn mower

 11 mushrooms

 12 pears

 13 pickup truck

 14 poppies

 15 possum

 16 ray

 17 skyscraper

 18 wardrobe

 19 whale

 0 beaver

 1 bowls

 2 boy

 3 bridge

 4 castle

 5 cockroach

 6 couc

KeyboardInterrupt: 

In [ ]:
for 